In [ ]:
import cv2
import numpy as np
from sklearn.cluster import DBSCAN
import os
from collections import defaultdict

class ImageProcessor:
    def __init__(self,
                 roi_coords: tuple = (0, 1280, 530, 720),
                 detect_corners_block_size: int = 7,
                 detect_corners_ksize: int = 3,
                 detect_corners_k: float = 0.09,
                 threshold_ratio: float = 0.01,
                 group_keypoints_dbscan_eps: float = 13,
                 group_keypoints_dbscan_min_samples: int = 60):
        """
        roi_coords: (x_start, x_end, y_start, y_end)
        Các tham số khác dùng cho Harris Corner Detection và DBSCAN clustering.
        """
        self.roi_coords = roi_coords
        self.detect_corners_block_size = detect_corners_block_size
        self.detect_corners_ksize = detect_corners_ksize
        self.detect_corners_k = detect_corners_k
        self.threshold_ratio = threshold_ratio
        self.group_keypoints_dbscan_eps = group_keypoints_dbscan_eps
        self.group_keypoints_dbscan_min_samples = group_keypoints_dbscan_min_samples

    def load_image(self, image_path: str) -> np.ndarray:
        """Tải ảnh từ đường dẫn."""
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Không thể tải ảnh từ: {image_path}")
        return image

    def extract_roi(self, image: np.ndarray) -> np.ndarray:
        """Cắt vùng ROI theo roi_coords."""
        x_start, x_end, y_start, y_end = self.roi_coords
        return image[y_start:y_end, x_start:x_end]

    def process_roi(self, roi: np.ndarray) -> np.ndarray:
        """
        Tiền xử lý ROI:
          - Chuyển ảnh sang grayscale.
          - (Có thể thêm CLAHE, threshold, morphology nếu cần)
        """
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        # Áp dụng Gaussian Blur để giảm nhiễu (cải thiện hiệu quả Canny)
        blurred = cv2.GaussianBlur(gray_roi, (5, 5), 1.4)

        # Áp dụng Canny edge detection
        # Tham số thứ 2 và thứ 3 là ngưỡng thấp và ngưỡng cao, có thể điều chỉnh để phù hợp với ảnh của bạn.
        edges = cv2.Canny(blurred, 50, 150)
        cv2.imshow("edges", edges)
        
        return edges

    def detect_corners(self, processed_img: np.ndarray) -> tuple:
        """
        Sử dụng thuật toán Harris để phát hiện góc và tạo mặt nạ.
        """
        corners = cv2.cornerHarris(np.float32(processed_img),
                                   self.detect_corners_block_size,
                                   self.detect_corners_ksize,
                                   self.detect_corners_k)
        threshold_value = self.threshold_ratio * corners.max()
        _, corner_mask = cv2.threshold(corners, threshold_value, 255, cv2.THRESH_BINARY)
        return corners, np.uint8(corner_mask)
    
    def group_and_fill(self, binary_image: np.ndarray, raw_image: np.ndarray, min_area: int = 5) -> np.ndarray:
        """
        Nhận diện các nhóm điểm ảnh liền kề, tìm tâm của mỗi nhóm và tô màu lên ảnh.
        
        Parameters:
        - image: Ảnh nhị phân đầu vào (0 và 255).
        - min_area: Ngưỡng diện tích tối thiểu để giữ lại nhóm.

        Returns:
        - result_img: Ảnh đã tô màu các nhóm điểm ảnh.x
        - centers: Danh sách tọa độ tâm của các nhóm [(x1, y1), (x2, y2), ...].
        """
        
        # Tìm các thành phần kết nối
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_image, connectivity=8)

        centers = []  # Lưu danh sách tâm của các nhóm

        for i in range(1, num_labels):  # Bỏ qua nhãn 0 (background)
            area = stats[i, cv2.CC_STAT_AREA]
            
            if area >= min_area:  # Chỉ xử lý nhóm có diện tích đủ lớn
                cx, cy = int(centroids[i][0]), int(centroids[i][1])  # Tọa độ tâm
                centers.append((cx, cy))
                
                # Vẽ tâm bằng hình tròn đỏ
                cv2.circle(raw_image, (cx, cy), 5, (0, 0, 255), -1)

        cv2.imshow("result_img", raw_image)

        return raw_image, centers


    def group_points_by_y(points, threshold=4):
        points.sort(key=lambda p: p[1])  # Sắp xếp theo y
        
        groups = defaultdict(list)
        group_index = 0
        prev_y = points[0][1]
        
        for x, y in points:
            if abs(y - prev_y) > threshold:
                group_index += 1  # Tạo nhóm mới nếu y khác biệt đáng kể
            groups[group_index].append((x, y))
            prev_y = y
        
        return list(groups.values())


    def filter_rows(self, rows: list) -> list:
        """
        Lọc bỏ các hàng có số lượng keypoints nhỏ (ít hơn 2).
        Nếu có quá nhiều hàng, bỏ hàng đầu và hàng cuối.
        """
        rows_filtered = [row for row in rows if len(row) > 1]
        if len(rows_filtered) > 2:
            rows_filtered = rows_filtered[1:-1]
        return rows_filtered

    def draw_keypoints_and_lines(self, image: np.ndarray, rows: list) -> np.ndarray:
        """
        Vẽ các keypoints và đường nối (màu xanh) lên ảnh gốc.
        """
        result = image.copy()
        for row in rows:
            row_sorted = sorted(row, key=lambda p: p[1])
            # Vẽ các đường nối
            for i in range(len(row_sorted) - 1):
                pt1 = (row_sorted[i][1], row_sorted[i][0])
                pt2 = (row_sorted[i + 1][1], row_sorted[i + 1][0])
                cv2.line(result, pt1, pt2, (255, 0, 0), 2)
            # Vẽ keypoints
            for y, x in row_sorted:
                cv2.circle(result, (x, y), 5, (0, 255, 0), -1)
        return result

    def error_checking(self, image: np.ndarray, rows: list, max_dist_threshold: float = 60, angle_threshold: float = 15) -> tuple:
        """
        Kiểm tra lỗi giữa các keypoints liên tiếp trong mỗi hàng theo khoảng cách và góc lệch.
        Vẽ đường nối màu đỏ nếu có lỗi và trả về danh sách lỗi.
        """
        final_result = image.copy()
        errors = []
        for row_idx, row in enumerate(rows):
            row_sorted = sorted(row, key=lambda p: p[1])
            for i in range(len(row_sorted) - 1):
                y1, x1 = row_sorted[i]
                y2, x2 = row_sorted[i + 1]
                dist = np.hypot(x2 - x1, y2 - y1)
                angle = np.degrees(np.arctan2((y2 - y1), (x2 - x1)))
                line_color = (255, 0, 0)
                if dist > max_dist_threshold or abs(angle) > angle_threshold:
                    line_color = (0, 0, 255)
                    errors.append(f"Hàng {row_idx}, cặp điểm {i}-{i+1}: khoảng cách {dist:.2f}, góc {angle:.2f}°")
                cv2.line(final_result, (x1, y1), (x2, y2), line_color, 2)
            for y, x in row_sorted:
                cv2.circle(final_result, (x, y), 5, (0, 255, 0), -1)
        return final_result, errors

    def process(self, src, isLoadImg: bool = False) -> tuple:
        """
        Pipeline xử lý ảnh:
          1. Tải ảnh (nếu src là đường dẫn).
          2. Cắt ROI và tiền xử lý.
          3. Phát hiện góc bằng Harris.
          4. Tìm keypoints, gom nhóm (DBSCAN) và điều chỉnh tọa độ.
          5. Nhóm keypoints theo hàng, vẽ các đường nối và kiểm tra lỗi.
        Trả về: (harris_display, image_result, final_result, errors)
        """
        image = src if isLoadImg else self.load_image(src)
        roi = self.extract_roi(image)
        processed_roi = self.process_roi(roi)
        corners, corner_mask = self.detect_corners(processed_roi)
        center_image, centers = self.group_and_fill(corner_mask,roi)
        cv2.imshow("center_image", center_image)
        print(centers)

        rows = self.group_points_by_y()
        # rows = self.group_rows(adjusted_keypoints)
        # filtered_rows = self.filter_rows(rows)
        # image_result = self.draw_keypoints_and_lines(image, rows)
        # final_result, errors = self.error_checking(image, filtered_rows)
        return 0,0,0,0, rows

    def processImgFolder(self, folderPath: str) -> None:
        """
        Duyệt qua tất cả các file ảnh trong folder, xử lý từng ảnh và hiển thị kết quả.
        """
        if not os.path.isdir(folderPath):
            print(f"Lỗi: Thư mục '{folderPath}' không tồn tại!")
            return

        image_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff')
        image_files = [f for f in os.listdir(folderPath) if f.lower().endswith(image_extensions)]
        if not image_files:
            print("Không tìm thấy ảnh trong thư mục!")
            return

        for img_name in image_files:
            img_path = os.path.join(folderPath, img_name)
            try:
                image = self.load_image(img_path)
            except ValueError as e:
                print(e)
                continue

            harris_disp, img_result, final_result, errors = self.process(image, isLoadImg=True)
            cv2.imshow(f"Corners: {img_name}", harris_disp)
            cv2.imshow(f"Final Result: {img_name}", final_result)
            print(f"Đang hiển thị ảnh: {img_name}")
            if errors:
                print("Các lỗi phát hiện:", errors)
            key = cv2.waitKey(0)
            if key & 0xFF == ord('q'):
                print("Thoát chương trình.")
                break
            cv2.destroyAllWindows()

    def realTime(self) -> None:
        """
        Xử lý thời gian thực từ camera.
        """
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print("Lỗi: Không mở được camera!")
            return

        self.roi_coords = (0, 1280, 475, 720)
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Lỗi: Không lấy được frame từ camera!")
                break
            frame = cv2.resize(frame, (1280, 720))
            harris_disp, img_result, final_result, errors = self.process(frame, isLoadImg=True)
            cv2.imshow("Camera - Keypoints & Lines", harris_disp)
            cv2.imshow("Camera - Error Checking", final_result)
            if errors:
                for err in errors:
                    print(err)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    folder_path = r"C:\Users\dinhk\OneDrive\Pictures\Camera Roll\60_thieu_cm"  # Thay đổi đường dẫn phù hợp
    processor = ImageProcessor()
    processor.processImgFolder(folder_path)
    # processor.realTime()
